In [1]:
import pandas as pd 

games = pd.read_csv("original_game.csv", header=1)

games

# check the bottom - may have a bunch of mostly-NaN rows

,auto?,# Players,# Cycles,# Posts,Fundamentals,Winner,Complexity,Broken,Setting,World (optional),...,# M killed,# N killed,# V killed,# FF,# Xed.1,# I killed,# Spectators,# pinchhitters?,GM? IM?,Anon?
0,LG1,16,4,462.0,V/E,E,Standard,Well Balanced,Cosmere,NaN,...,0,0,1,0,4,0,3,NaN,NaN,NaN
1,LG2,29,11,1232.0,Conversion,E,Standard,Balanced,Cosmere,NaN,...,0,0,2,2,7,0,2,NaN,NaN,NaN
2,LG3,22,7,739.0,V/E,V,Standard,Broken Distribution,Cosmere,NaN,...,0,0,1,3,7,0,5,NaN,NaN,NaN
3,LG4,30,9,1029.0,V/E,V,Semi Standard,Broken Mechanics,Cosmere,NaN,...,0,1,4,4,8,0,1,NaN,NaN,NaN
4,QF1,12,4,118.0,V/E,E,Basic,Well Balanced,Cosmere,NaN,...,0,0,0,0,3,0,11,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,LG82,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,2,0,0,NaN,NaN,NaN
203,QF57,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,3,0,0,NaN,NaN,NaN
204,MR55,12,NaN,NaN,V/E,NaN,Complex,NaN,NaN,NaN,...,0,0,0,1,4,0,0,NaN,NaN,NaN
205,AG8,29,NaN,NaN,V/E,V,Standard,NaN,Cosmere,Scadrial Era 1,...,0,0,0,0,8,1,0,NaN,NaN,NaN


In [2]:
games.rename(inplace=True, columns={
    "auto?": "game_string",
    "# Players": "num_players",
    "# Cycles": "num_cycles",
    "# Posts": "num_posts",
    "Fundamentals": "fundamentals", 
    "Winner": "winner",
    "Complexity": "complexity",
    "Broken": "broken",
    "Setting": "setting",
    "World (optional)": "world",
    "Anon?": "is_anon"
    })

# remaining columns are just calculated from data test
games = games.filter([
    "game_string", 
    "num_cycles", 
    "num_posts", 
    "fundamentals", 
    "winner", 
    "complexity", 
    "broken", 
    "setting", 
    "world",
    "is_anon"
])

# really would love to know how to keep int columns as integers 

games.head()

,game_string,num_cycles,num_posts,fundamentals,winner,complexity,broken,setting,world,is_anon
0,LG1,4,462.0,V/E,E,Standard,Well Balanced,Cosmere,NaN,NaN
1,LG2,11,1232.0,Conversion,E,Standard,Balanced,Cosmere,NaN,NaN
2,LG3,7,739.0,V/E,V,Standard,Broken Distribution,Cosmere,NaN,NaN
3,LG4,9,1029.0,V/E,V,Semi Standard,Broken Mechanics,Cosmere,NaN,NaN
4,QF1,4,118.0,V/E,E,Basic,Well Balanced,Cosmere,NaN,NaN


In [3]:
# not sure whether to do this or put above inside game part of table_splitting 
game_partial = pd.read_csv("game_partial.csv", index_col=0)

game_partial.rename(inplace=True, columns={"Broken": "is_broken"})

game_partial

,format,game_num,game_string,is_broken,game_id,IM_id
0,LG,1.0,LG1,NaN,1,NaN
1,LG,2.0,LG2,NaN,2,NaN
2,LG,3.0,LG3,Y,3,NaN
3,LG,4.0,LG4,Y,4,NaN
4,QF,1.0,QF1,NaN,5,NaN
...,...,...,...,...,...,...
204,LG,82.0,LG82,NaN,204,253.0
205,QF,57.0,QF57,NaN,205,236.0
206,MR,55.0,MR55,NaN,206,65.0
207,AG,8.0,AG8,NaN,207,12.0


In [1]:
games = games.merge(game_partial, how="outer", on="game_string")

games

NameError: name 'games' is not defined

In [ ]:
games.to_csv("game.csv")

Inserting into SQL

In [ ]:
import psycopg2

db=1

In [13]:
row = {
    "game_id": "1", 
    "game_format": "LG", 
    "game_number": "25", 
    "game_string": "LG25", 
    "is_anon": "N",
    "broken": "Y",
    "IM_id": "27", 
    "world": "Nalthis",
    "complexity": "Very", # mmm
    "fundamentals": "Simple"
    }

In [14]:
# ? rows in iterrows are copies, right, we're not changing the original?
def insertGame(row):
    insert = "insert into game (game_id, game_format, game_number, game_string, is_anon, broken, IM_id, world, complexity, fundamentals) "
    values = "values ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');"

    # CLEANING THAT COULD GO ELSEWHERE------------
    for key in row.keys():
        if row[key] == "NaN": # quotes are fine right? 
            row[key] = "null"
    
    # ew
    if row["is_anon"] == "Y":
        row["is_anon"] = "true"
    else:
        row["is_anon"] = "false"

    if row["broken"] == "Y":
        row["broken"] = "true"
    else:
        row["broken"] = "false"
    # ------------------------------------------

    ret = insert + values.format(row["game_id"],
        row["game_format"], 
        row["game_number"], 
        row["game_string"], 
        row["is_anon"],
        row["broken"],
        row["IM_id"], # that's gonna be a problem potentially 
        row["world"],
        row["complexity"],
        row["fundamentals"])

    return ret

In [15]:
insertGame(row)

Y


"insert into game (game_id, game_format, game_number, game_string, is_anon, broken, IM_id, world, complexity, fundamentals) values ('1', 'LG', '25', 'LG25', 'false', 'true', '27', 'Nalthis', 'Very', 'Simple');"

In [ ]:
if db!=0:
    cnx = psycopg2.connect(user="isanford_123", database="isanford", host="localhost", password="12345678")
    cursor = cnx.cursor()


game_string	
num_cycles	
num_posts	
fundamentals	
winner	
complexity	
broken	
setting	
world	
is_anon	
format	
game_num	
is_broken	
game_id	
IM_id

In [ ]:
cc = 0

for row in games.iterrows():
    # check for NaN row?
    ins = insertGame(row[1])

    if db == 1:
        try:
            cursor.execute(ins)
        except:
            print(row)
            print(ins)
            cc = cc + 1 # error count??
    else:
        print(ins)

print(f"Num errors: {cc}")

In [ ]:

cc=0
with open("hurdat2-1851-2019-052520.txt") as fp:
    line = fp.readline()
    hid=''
    while line:
        spl = line.strip().split(",")
        if len(spl)<6:
            hid=spl[0]
            ins = insertStringH(spl)
        else:
            ins = insertStringO(hid, spl)
        if db==1:
            try:
                cursor.execute(ins)
            except:
                print(line)
                print(ins)
                cc=cc+1
        else:
            print(ins)
        line = fp.readline()

print(cc)